In [1]:
# !pip install evaluate==0.4.3
# Остальные зависимости в requirements.txt

In [2]:
import json
import os
import re
import time
from typing import List

import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from dotenv import load_dotenv
from evaluate import load
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from openai import OpenAI

load_dotenv()

True

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Device: cuda


### Зададим константы

In [4]:
# Датасет с Hugging Face
DATASET = "neural-bridge/rag-dataset-1200"
SPLIT_DATASET = "test"

# Модели
    # Загружаем модель эмбедингов
EMBEDDING_MODEL = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={"device": device}
)

# Определяем модель для генерации ответа на основе документов
LLM_MODEL = "meta-llama/llama-3.2-3b-instruct:free"

# API-конфигурация к LLM
CLIENT = OpenAI(
    base_url="https://openrouter.ai/api/v1", 
    api_key=os.getenv("TOKEN_OPENAI")
)

BERTSCORE = load("bertscore")

# Создадим два промпта для уменьшения вероятности нерелевантного ответа

    # Промпт для LLM, который просит определить только релевантные документы
DOC_RETRIEVAL_PROMPT = (
    "You are an AI assistant specialized in document retrieval. "
    "Your task is to extract only the most relevant document IDs and chunk IDs from the provided documents. "
    "Strictly follow these rules: "
    "1. Return only a JSON object in this exact format: "
    '{"relevant_documents": [{"document_id": <doc_id>, "chunk_id": <chunk_id>}, ...]}. '
    "2. Do not modify, summarize, or explain the documents. "
    "3. Do not include any additional text, explanations, reasoning, or commentary. "
    "4. Do not return the document content, only IDs. "
    "5. If no relevant documents exist, return an empty JSON: {\"relevant_documents\": []}. "
    "6. Any deviation from these rules is strictly prohibited."
)

    # Промпт для LLM, который просит составить ответ только на релевантных документах
ANSWER_GENERATION_PROMPT = (
    "You are an assistant that answers user questions based strictly on the provided documents. "
    "Use only the content from the relevant documents and chunks listed below: "
    "{retrieved_data} "
    "Now, generate a well-structured answer to the user's question."
    "Do not make up information. If the answer is unclear from the documents, say 'Insufficient information'."
)

### Определим данные и создадим базу знаний

In [5]:
# Загружаем данные и смотрим что в них
rag_dataset = load_dataset(DATASET, split=SPLIT_DATASET)
display(rag_dataset)

# Смотрим пример одного row
print('Dataset example:') 
rag_dataset[0]

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 240
})

Dataset example:


{'context': 'Trail Patrol Training\nWant to be a part of the Trail Patrol ?? Join an Orientation & Hike on the 1st Tuesday of each month. This course is required for all PATC members interested in joining the PATC Trail Patrol.\nThe course teaches the essential skills necessary to be a trail patrol member and to provide a reassuring presence on the trail while teaching safety and environmental responsibility. A Trail Patrol handbook is provided to all students. Please bring a pencil, your hiking daypack & lunch.\nMore Info: View the Calendar or contact TP Training or visit the Trail Patrol Training web pages.\nHike Leader Class.\nMore Info: Contact Hike Leader Training or click here to register.\nBackpacking Classes\nEducating people in safe and environmentally friendly practices for traveling into the backcountry is one of Trail Patrol’s core responsibilities. We offer backpacking classes for novices seeking to take up backpacking as well as for experienced backpackers.\nBackpacking 1

In [6]:
def chunk_documents(documents: List[str], chunk_size: int = 1000, chunk_overlap: int = 100) -> List[Document]:
    """
    Разбивает документы на чанки.

    Args:
        documents (List[Document]): Список документов.
        chunk_size (int): Размер чанка в символах.
        chunk_overlap (int): Перекрытие чанков.

    Returns:
        List[Document]: Разбитые на чанки документы.
    """
    print(f"Разбиваем {len(documents)} документов на чанки (размер: {chunk_size}, перекрытие: {chunk_overlap})")
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunked_documents = []

    # Предобработка всех текстовых файлов\документов\текстов
    for i, document in enumerate(documents):
        # Минимально очистим текст
        text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', document)  # Удаляем лишние пустые строки
        text = re.sub(r'[ \t]+', ' ', text)  # Заменяем табуляции на пробелы
        text = text.strip()

        # Создадим langchain-документ (совместимый формат для RecursiveCharacterTextSplitter)
        langchain_document = Document(page_content=text)
        # Разобьём длинный текст на чанки
        chunks = text_splitter.split_documents([langchain_document])

        # Добавим номер документа (текста) и номер чанка как доп.информацию
        # И добавим сам чанк в список всех чанков
        for j, chunk in enumerate(chunks):
            chunk.metadata["document_id"] = i + 1
            chunk.metadata["chunk_id"] = j + 1
            chunked_documents.append(chunk)

    print(f"Создано {len(chunked_documents)} чанков")
    return chunked_documents

In [7]:
def create_retriever(documents: List[Document], weights: List[float] = [0.4, 0.6]) -> EnsembleRetriever:
    """
    Создаёт EnsembleRetriever на основе FAISS и BM25.

    Args:
        documents (List[Document]): Список документов для индексации.
        weights (List[float]): Веса для каждого из retrievers

    Returns:
        EnsembleRetriever: Комбинированный ретривер для поиска.
    """
    # FAISS retriever
    vector_store = FAISS.from_documents(documents, EMBEDDING_MODEL)
    faiss_retriever = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={'k': 2}  # Количество возвращаемых документов
    )

    # BM25 retriever
    bm25_retriever = BM25Retriever.from_documents(documents)
    bm25_retriever.k = 2  # Количество возвращаемых документов

    # Объединяем их в EnsembleRetriever
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, faiss_retriever],
        weights=weights  # Вес каждого retriever
    )

    return ensemble_retriever

In [8]:
chunked_documents = chunk_documents(rag_dataset['context'])
chunked_documents[3]

Разбиваем 240 документов на чанки (размер: 1000, перекрытие: 100)
Создано 1139 чанков


Document(metadata={'document_id': 1, 'chunk_id': 4}, page_content='More Info: Contact Backpacking Training or click here to register.\nWilderness First Aid\nBasic Wilderness First Aid (BWFA) is a 2-day workshop. Day one covers Adult CPR and AED and American Heart Association First Aid. You will receive a textbook and a certification card good for two years. Day two is American Safety and Health Institute (ASHI) Basic Wilderness First Aid. You will learn how to do bleeding control, splinting and other basic first aid skills in the wilderness setting. There is plenty of hands-on time & paramedics with years of backcountry experience teach the classes.\nMore Info: Contact TP First Aid or click here to register.\nWilderness First Aid (WFA)')

In [9]:
ensemble_retriever = create_retriever(chunked_documents)

### Убедимся что всё работает на тестовом примере

In [10]:
# Пример запроса
test_query = rag_dataset['question'][3]
test_query

'Who were the two convicted killers that escaped from an upstate New York maximum-security prison?'

In [12]:
def get_llm_response(system_prompt: str, docs: str, query: str, temperature: float = 0.0) -> str:
    """
    Отправляет запрос в LLM, используя заданный системный промпт.

    Args:
        system_prompt (str): Промпт, определяющий задачу для модели.
        docs (str): Документы в формате JSON, содержащие релевантную информацию.
        query (str): Вопрос пользователя.
        temperature (float): Температура ответа (насколько модель креативна)

    Returns:
        str: Ответ LLM.
    """
    # Формируем контекст для LLM
    chat_history = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'documents', 'content': docs},
        {'role': 'user', 'content': query}
    ]


    response = CLIENT.chat.completions.create(
        model=LLM_MODEL,
        messages=chat_history,
        temperature=temperature,
        max_tokens=2048
    ).choices[0].message.content
    
    return response

In [11]:
# Поиск релевантных документов
relevant_docs = ensemble_retriever.invoke(test_query)

# Преобразуем найденные документы в нужный нам формат
relevant_docs_data = [
    {
        "document_id": doc.metadata.get("document_id", -1),
        "chunk_id": doc.metadata.get("chunk_id", -1),
        "content": doc.page_content
    }
    for doc in relevant_docs
]
json_docs = json.dumps(relevant_docs_data, ensure_ascii=False)

In [13]:
# Первый запрос к LLM: получение списка ID релевантных документов
id_docs_response = get_llm_response(
    system_prompt=DOC_RETRIEVAL_PROMPT, 
    docs=json_docs, 
    query=test_query,
    temperature=0.0 # нам нужен наиболее строгий ответ
)
print(f"Полученные идентификаторы документов: {id_docs_response}")

Полученные идентификаторы документов: {"relevant_documents": [{"document_id": 4, "chunk_id": 1}]}


In [14]:
# Второй запрос к LLM: генерация финального ответа
response = get_llm_response(
    system_prompt=ANSWER_GENERATION_PROMPT.format(retrieved_data=id_docs_response), 
    docs=json_docs, 
    query=test_query,
    temperature=0.3 # можем быть слегка креативными
)
response

'The two convicted killers who escaped from an upstate New York maximum-security prison were Richard Matt and David Sweat.'

In [15]:
rag_dataset[3]['answer']

'The two convicted killers that escaped from an upstate New York maximum-security prison were Richard Matt and David Sweat.'

### Проверим насколько качественно работает наша RAG система

In [21]:
# Положим вопросы и ответы в датафрейм и возьмём лишь 20 примеров (т.к. API имеет лимиты)
df = pd.DataFrame({
    'question': rag_dataset['question'],
    'answer': rag_dataset['answer'],
})
df_sample = df.sample(20, random_state=42)
df_sample.reset_index(drop=True, inplace=True)
df_sample.head()

,question,answer
24,What are some of the features of the yellow pa...,The yellow paper plates mentioned in the conte...
6,"What is a ""Cultural Muslim"" according to Kaigh...","A ""Cultural Muslim"" is someone who calls thems..."
93,"Who are the main characters in the book ""Odd a...","The main characters in the book ""Odd and the F..."
109,What are some features of the WHIRLPOOL BATHTU...,The WHIRLPOOL BATHTUB model AM152JDTS-1Z has s...
104,What is the rank of Iasi in terms of populatio...,Iasi is the 4th biggest city in Romania in ter...


In [23]:
def rag_answer(query):
    """
    Генерирует ответ на заданный запрос, используя retriever, документы и LLM.

    Аргументы:
        query (str): Вопрос пользователя.

    Возвращает:
        str: Сгенерированный ответ модели.
    """
    # Поиск релевантных документов
    relevant_docs = ensemble_retriever.invoke(query)
    relevant_docs_data = [
        {
            "document_id": doc.metadata.get("document_id", -1),
            "chunk_id": doc.metadata.get("chunk_id", -1),
            "content": doc.page_content
        }
        for doc in relevant_docs
    ]
    json_docs = json.dumps(relevant_docs_data, ensure_ascii=False)
    
    # Обращения к LLM
    id_docs_response = get_llm_response(
        system_prompt=DOC_RETRIEVAL_PROMPT, 
        docs=json_docs, 
        query=query,
        temperature=0.0
    )
    
    response = get_llm_response(
        system_prompt=ANSWER_GENERATION_PROMPT.format(retrieved_data=id_docs_response), 
        docs=json_docs, 
        query=query,
        temperature=0.3
    )
    
    # Нужно из-за ограничений API к LLM
    time.sleep(17)
    return response

In [24]:
df_sample['rag_answer'] = df_sample['question'].apply(rag_answer)
df_sample.head()

,question,answer,rag_answer
24,What are some of the features of the yellow pa...,The yellow paper plates mentioned in the conte...,"Based on the provided documents, some of the f..."
6,"What is a ""Cultural Muslim"" according to Kaigh...","A ""Cultural Muslim"" is someone who calls thems...","According to the documents, a ""Cultural Muslim..."
93,"Who are the main characters in the book ""Odd a...","The main characters in the book ""Odd and the F...",Insufficient information. The provided documen...
109,What are some features of the WHIRLPOOL BATHTU...,The WHIRLPOOL BATHTUB model AM152JDTS-1Z has s...,"According to the provided documents, the WHIRL..."
104,What is the rank of Iasi in terms of populatio...,Iasi is the 4th biggest city in Romania in ter...,"According to the documents, Iasi is the 4th bi..."


In [51]:
def compute_bertscore(predictions: List[str], references: List[str], lang: str = "en") -> float:
    """
    Вычисляет средний F1-скор BertScore между предсказанными и эталонными ответами.

    Аргументы:
        predictions (List[str]): Список сгенерированных моделью ответов.
        references (List[str]): Список эталонных (правильных) ответов.
        lang (str): Язык текстов (по умолчанию "en").

    Возвращает:
        float: Средний F1 из BertScore.
    """
    results = BERTSCORE.compute(predictions=predictions, references=references, lang=lang)
    return float(np.mean(results["f1"]))


In [52]:
avg_f1 = compute_bertscore(df_sample['rag_answer'].tolist(), df_sample['answer'].tolist())
print(f"BertScore F1: {avg_f1:.4f}")

BertScore F1: 0.9278
